In [4]:
!pip install -q qdrant-client
!pip install -q sentence-transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.1 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.1 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires protobuf<4.0.0dev,>=3.12.0, but you have protobuf 4.25.1 which is incompatible.
google-cloud-bigtable 1.7.3 requires protobuf<4.0.0dev, but you have protobuf 4.25.1 which is incompatible.
google-cloud-datastore 

In [16]:
!pip install -q torch

In [19]:
import warnings
warnings.filterwarnings('ignore')

from transformers import pipeline
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm.auto import tqdm
# from typing import List
import torch

In [1]:
import pickle
import numpy as np

In [2]:
VECTOR_SPACE_PATH = "/kaggle/input/chaabi/space.pkl"
with open(VECTOR_SPACE_PATH, 'rb') as file:
    vs = pickle.load(file)

In [3]:
vs["payload"][0]

{'id': 0,
 'product': 'Garlic Oil - Vegetarian Capsule 500 mg',
 'story': 'Garlic Oil - Vegetarian Capsule 500 mg is of category Beauty & Hygiene and sub category is Hair Care. Garlic Oil - Vegetarian Capsule 500 mg is type Hair Oil & Serum. brand of Garlic Oil - Vegetarian Capsule 500 mg is Sri Sri Ayurveda , with rating 4.1. sale price of Garlic Oil - Vegetarian Capsule 500 mg is 220.0 with market price 220.0, description of Garlic Oil - Vegetarian Capsule 500 mg is This Product contains Garlic Oil that is known to help proper digestion, maintain proper cholesterol levels, support cardiovascular and also build immunity.  For Beauty tips, tricks & more visit https://bigbasket.blog/'}

In [6]:
client = QdrantClient(":memory:")

In [7]:
collection_name = "extractive-question-answering"
collection_name = "qdrant-space"

collections = client.get_collections()
if collection_name not in [c.name for c in collections.collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
        ),
    )
print(client.get_collections())

collections=[CollectionDescription(name='qdrant-space')]


In [8]:
%%time

 
total_records = len(vs["payload"]) # total records data
_payload = vs["payload"]
_emb = vs["emb"]
ids = list(range(0,total_records))

batch_size = 2  # specify batch size according to your RAM and compute, higher batch size = more RAM usage

client.upsert(
    collection_name=collection_name,
    points=models.Batch(ids=ids, vectors=_emb, payloads=_payload),
)


collection_vector_count = client.get_collection(collection_name=collection_name).vectors_count
print(f"Vector count in collection: {collection_vector_count}")


Vector count in collection: 27555
CPU times: user 2min 4s, sys: 478 ms, total: 2min 4s
Wall time: 2min 4s


In [20]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
    
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
# retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
retriever


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [9]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

# load the reader model into a question-answering pipeline
reader = pipeline("question-answering", model=model_name, tokenizer=model_name)
print(reader.model, reader)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

In [13]:
def find_close_contexts(question: str, top_k: int) -> List[str]:
    """
    will return contexts contexts close to query 

    Args:
        question (str): What do we want to know?
        top_k (int): top k results will be added 

    Returns:
        context (List[str]):
    """
    try:
        encoded_query = retriever.encode(question).tolist() 
        result = client.search(
            collection_name=collection_name,
            query_vector=encoded_query,
            limit=top_k,
        )  # search qdrant collection for context passage with the answer

        context = [
            [context.payload["product"], context.payload["story"]] for context in result
        ] 
        return context
    except Exception as e:
        print({e})

In [25]:
question = "what is price of Garlic Oil - Vegetarian Capsule 500 mg"
context = find_close_contexts(question, top_k=1)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [96]:
def tell_me(question: str, context: List[str]):
    """
    Extract the answer from the context for a given question

    Args:
        question (str): _description_
        context (list[str]): _description_
    """
    results = []
    for c in context:
        answer = reader(question=question, context=c[1] )
        answer["product"] = c[0]
        results.append(answer)
        print()

    sorted_result = sorted(results, key=lambda x: x["score"], reverse=True)
    for i in range(len(sorted_result)):
        _out = sorted_result[i]["answer"] 
        _prod = sorted_result[i]["product"]
        _sco = sorted_result[i]["score"]
#         print(f"{i+1}", end=" ")
        print(f"QUERY INPUT: {question}")
        print(f"OUTPUT: {_out} \nPREDICTION SCORE {_sco}\n\nReferred Product: {_prod}\n\n  ")




In [107]:
queries = []
queries.append("suggest me one product for cleaning vegetables")
queries.append("what is the rating of product Vegetable & Fruit Wash with 100% Natural Action")
queries.append("give me soap brand name")
queries.append("")



In [108]:
for q in queries:
    c = find_close_contexts(q, top_k=1)
    tell_me(q,c)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


QUERY INPUT: suggest me one product for cleaning vegetables
OUTPUT: Vegetable & Fruit Wash 
PREDICTION SCORE 0.003299231640994549

Referred Product: Vegetable & Fruit Wash with 100% Natural Action

  


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


QUERY INPUT: what is the rating of product Vegetable & Fruit Wash with 100% Natural Action
OUTPUT: 4.3 
PREDICTION SCORE 0.9139595627784729

Referred Product: Vegetable & Fruit Wash with 100% Natural Action

  


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


QUERY INPUT: give me soap brand name
OUTPUT: Nyassa 
PREDICTION SCORE 0.7516050338745117

Referred Product: Oudh Golab Middle Eastern Handmade Soap

  
